In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer, PorterStemmer


with open('/Users/alexbean/Desktop/tfidf_data.pkl', 'rb') as file:
    tfidf = pickle.load(file)
with open('/Users/alexbean/Desktop/X_train_preproc.pkl', 'rb') as file:
    X_train_preproc = pickle.load(file)
with open('/Users/alexbean/Desktop/X_val_preproc.pkl', 'rb') as file:
    X_val_preproc = pickle.load(file)
with open('/Users/alexbean/Desktop/X_test_preproc.pkl', 'rb') as file:
    X_test_preproc = pickle.load(file)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


train_data = pd.read_csv('./archive/train.csv', header=None)
train_data.columns=['polarity', 'title', 'text',]

val_data = pd.read_csv('./archive/val.csv', header=None)
val_data.columns=['polarity', 'title', 'text',]

test_data = pd.read_csv('./archive/test.csv')

In [ ]:
X_train   = train_data['text']
y_train   = train_data['polarity']

# get val data
X_val    = val_data['text']
y_val    = val_data['polarity']

# get test data
X_test   = test_data['text']
y_test   = test_data['polarity']

print(f"Train Data Shape: {X_train.shape[0]:,}")
print(f"Validation Data Shape: {X_val.shape[0]:,}")
print(f"Test Data Shape: {X_test.shape[0]:,}")

print(" ")
print(f"Number of labels = 1 in train dataset as percentage: {((y_train == 1).sum() / (X_train.shape[0])) * 100:0.2f}%")
print(f"Number of labels = 2 in train dataset as percentage: {((y_train == 2).sum() / (X_train.shape[0])) * 100:0.2f}%")

print(" ")
print(f"Number of labels = 1 in val dataset as percentage: {((y_val == 1).sum() / (X_val.shape[0])) * 100:0.2f}%")
print(f"Number of labels = 2 in val dataset as percentage: {((y_val == 2).sum() / (X_val.shape[0])) * 100:0.2f}%")

print(" ")
print(f"Number of labels = 1 in test dataset as percentage: {((y_test == 1).sum() / (X_test.shape[0])) * 100:0.2f}%")
print(f"Number of labels = 2 in test dataset as percentage: {((y_test == 2).sum() / (X_test.shape[0])) * 100:0.2f}%")

In [ ]:
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

sets_combined_x = pd.concat([X_train_preproc, X_val_preproc], ignore_index=True)
sets_combined_y = pd.concat([y_train, y_val], ignore_index=True)

In [ ]:
tfidf = TfidfVectorizer(strip_accents='unicode', lowercase=True, tokenizer=tokenizer_porter, stop_words='english', max_df=0.9)
X_train_tfidf = tfidf.fit_transform(sets_combined_x)
print(f"\nTF-IDF feature matrix shape: {X_train_tfidf.shape}")

In [ ]:
X_test_tfidf = tfidf.transform(X_test_preproc)
print(f"\nTF-IDF feature matrix shape: {X_test_tfidf.shape}")

In [ ]:
model = LogisticRegression(**{'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 100, 'C': 1, 'random_state': 42})
model.fit(X_train_tfidf, sets_combined_y)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(X_test_tfidf)

val_accuracy_lr = accuracy_score(y_test, y_pred) # Calculate validation accuracy

print("\n--- Logistic Regression (Validation) ---")
print(f"Validation Accuracy: {val_accuracy_lr:.4f}")
print(f"Validation F1 Score: {f1_score(y_test, y_pred, average='weighted'):.4f}")
print("Validation Classification Report:")
print(classification_report(y_test, y_pred))

In [2]:
# Extract feature names from the vectorizer
feature_names = tfidf.get_feature_names_out()

# Get coefficients from the logistic regression model
# Assuming model is a binary logistic regression with shape (1, n_features)
coefficients = model.coef_[0]

# Sort features by their coefficient values
# Negative coefficients are associated with negative class, positive with positive class
sorted_indices = np.argsort(coefficients)

# Adjust these numbers as needed (e.g., top 20)
top_negative_indices = sorted_indices[:20]  # 20 most negative words

# Get the corresponding words and their coefficients
top_negative_words = [(feature_names[i], coefficients[i]) for i in top_negative_indices]

# Print top negative words (aspects potentially causing negative feedback)
print("Top Negative Words:")
for word, coef in top_negative_words:
    print(f"{word}: {coef:.3f}")

NameError: name 'tfidf' is not defined

In [1]:
top_negative_words = [(feature_names[i], coefficients[i]) for i in top_negative_indices]

# Prepare a list to hold the contribution scores for each comment
comment_contributions = []

# Loop through the test dataset and calculate the contribution of top negative words
for i, comment in enumerate(X_test_preproc[y_test == 1].head(5)):
    # Get the TF-IDF scores for the current comment
    comment_tfidf = X_test_tfidf[i]
    
    # Calculate the total negative contribution of the comment (by summing contributions from top negative words)
    total_negative_contribution = 0
    for word, coef in top_negative_words:
        # Get the index of the word in the vectorizer's feature names
        if word in tfidf.get_feature_names_out():
            word_index = tfidf.get_feature_names_out().tolist().index(word)
            tfidf_score = comment_tfidf[0, word_index]
            
            # Contribution of this word to the sentiment (TF-IDF * Coefficient)
            contribution = tfidf_score * coef
            total_negative_contribution += contribution
    
    # Store the total negative contribution for this comment
    comment_contributions.append((i, total_negative_contribution))

# Sort the comments by their total negative contribution in descending order
comment_contributions.sort(key=lambda x: x[1], reverse=True)

NameError: name 'top_negative_indices' is not defined

In [ ]:
# Print the top 10 comments with the most negative contributions
print("Top 10 Comments with the Most Negative Words:")

for i, _ in comment_contributions[:10]:  # Get the top 10 comments
    print(f"Comment: {X_test[i]}")
    print(f"Negative Contribution: {comment_contributions[i][1]:.3f}")
    print("-" * 80)

In [ ]:
top_negative_comments = pd.DataFrame({
    'comment': [X_test[i] for i, _ in comment_contributions[:10]],  # Top 10 comments
    'negative_contribution': [comment_contributions[i][1] for i, _ in comment_contributions[:10]]
})